In [ ]:
import duckdb
import pandas as pd
import pyarrow.parquet as pq
from itertools import islice, tee

# from src.core import Data, Reader

In [35]:
d = "/home/anadirov/Documents/big.parquet"
d2 = "/home/anadirov/Documents/local_dwh/mp_all_offers_prod.parquet"
# pd_df = pd.read_parquet(d)

# q = f"""
#     create table data as select * from "{d}"
# """.strip()

In [38]:
df = duckdb.read_parquet(d)
df2 = duckdb.read_parquet(d2)

In [ ]:
# parquet_file = pq.ParquetFile(d)
# gen = parquet_file.iter_batches(batch_size=1000)

In [ ]:
def nth_item(generator, n):
    return next(islice(generator, n, n+1), None)

In [ ]:
def count_items(generator):
    return sum(1 for _ in generator)

def count_and_copy(generator):
    gen_copy, gen_count = tee(generator)
    count = sum(1 for _ in gen_count)
    return gen_copy, count


# count_and_copy(gen)

In [ ]:
# r = df.query("data", "select * from data limit 10000")

In [ ]:
# bats = r.to_arrow_table().to_batches(10)

In [ ]:
# big = Data(path = d, 
#            virtual_table_name = "data", 
#            query = "select item_id, __index_level_0__ from data order by __index_level_0__")

In [54]:
from io import StringIO
import sys, os

def render_df_info(duckdf: duckdb.DuckDBPyRelation) -> str:
    """ returns md like formatted df.info """
    shape = duckdf.shape

    output_buffer = StringIO()
    sys.stdout = output_buffer
    duckdf.describe().show(max_width=10**19)
    sys.stdout = sys.__stdout__
    descr = output_buffer.getvalue()

    h = f"### Rows: {shape[0]}, Columns: {shape[1]}\n{'-'*50}\n"
    try:
        lines = descr.strip().split("\n")
        headers = lines[1].strip("│").split("│")
        headers = [header.strip() for header in headers]
        types = lines[2].strip("│").split("│")
        types = [t.strip() for t in types]
        data_lines = lines[4:-1]
        data = []
        for line in data_lines:
            row = line.strip("│").split("│")
            row = [item.strip() for item in row]
            data.append(row)

        # Build the markdown table
        markdown_table = "| " + " | ".join(headers) + " |\n"
        markdown_table += "|-" + "-|-".join(["-" * len(header) for header in headers]) + "-|\n"
        markdown_table += "| " + " | ".join(types) + " |\n"
        for row in data:
            markdown_table += "| " + " | ".join(row) + " |\n"

        return h + markdown_table
    
    except Exception as e:
        return h


r = render_df_info(df2)